<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NCSES-class---FedRePORTER-and-IPEDS-data" data-toc-modified-id="NCSES-class---FedRePORTER-and-IPEDS-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NCSES class - FedRePORTER and IPEDS data</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Python Setup</a></span></li></ul></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span><ul class="toc-item"><li><span><a href="#Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)" data-toc-modified-id="Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Federal RePORTER - Abstracts (<a href="https://federalreporter.nih.gov/FileDownload" target="_blank">https://federalreporter.nih.gov/FileDownload</a>)</a></span></li><li><span><a href="#NMF-method---Non-negative-matrix-factorization" data-toc-modified-id="NMF-method---Non-negative-matrix-factorization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NMF method - Non-negative matrix factorization</a></span></li></ul></li></ul></div>

## NCSES class - FedRePORTER and IPEDS data

### Introduction

**Federal RePORTER** (https://federalreporter.nih.gov) - a collaborative effort led by STAR METRICS® to create a searchable database of scientific awards from agencies (across agencies or fiscal years, by the award's project leader, or by a text search of a project's title, terms, or abstracts).

### Python Setup

In [1]:
# Data manipulation
import pandas as pd

# Reading in files
import glob

# Text analysis (topic modeling)
import numpy as np
import sklearn
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import string

## Load the data

### Federal RePORTER - Abstracts (https://federalreporter.nih.gov/FileDownload)

In [2]:
"""Get all files with project abstracts."""

abstracts_files = glob.glob('FedRePORTER_PRJABS_C_FY20*.csv')
print(abstracts_files)

['FedRePORTER_PRJABS_C_FY2009.csv', 'FedRePORTER_PRJABS_C_FY2008.csv', 'FedRePORTER_PRJABS_C_FY2018.csv', 'FedRePORTER_PRJABS_C_FY2017.csv', 'FedRePORTER_PRJABS_C_FY2003.csv', 'FedRePORTER_PRJABS_C_FY2002.csv', 'FedRePORTER_PRJABS_C_FY2016.csv', 'FedRePORTER_PRJABS_C_FY2000.csv', 'FedRePORTER_PRJABS_C_FY2014.csv', 'FedRePORTER_PRJABS_C_FY2015.csv', 'FedRePORTER_PRJABS_C_FY2001.csv', 'FedRePORTER_PRJABS_C_FY2005.csv', 'FedRePORTER_PRJABS_C_FY2011.csv', 'FedRePORTER_PRJABS_C_FY2010.csv', 'FedRePORTER_PRJABS_C_FY2004.csv', 'FedRePORTER_PRJABS_C_FY2012.csv', 'FedRePORTER_PRJABS_C_FY2006.csv', 'FedRePORTER_PRJABS_C_FY2007.csv', 'FedRePORTER_PRJABS_C_FY2013.csv']


In [3]:
"""Read them in, concatenate and convert to a dataframe."""

list_data = []
for filename in abstracts_files:
    data = pd.read_csv(filename)
    list_data.append(data)
    
abstracts = pd.concat(list_data)

In [4]:
"""Drop missing abstracts."""

abstracts = abstracts.dropna()

In [5]:
"""Get abstracts as a list to feed in to TfidfVectorizer in the next step."""

merged_abstracts_list = abstracts[' ABSTRACT'].values.tolist()

### NMF method - Non-negative matrix factorization

NMF is a model used for topic extraction - while the LDA model uses raw counts of unique words per document, NMF model uses a normalized representation of those raw counts (TF-IDF representation)

TF stands for term-frequency and TF-IDF is term-frequency times inverse document-frequency. In other words, we are not only looking for how often a word appears in a given document, but also whether this particular word is distinct across all the collections of documents (corpus). For example, intuitively we understand that words like "often" or "use" are more frequently encountered, but they are less informative (more semantically-vacuous) if we want to discern a particular topic of a document, as they might be frequently encounter across all text documents in a corpus. On the other hand, words which we will see less frequently across a collection of document might indicate that those words are specific to a particular document, and, therefore, constitute a basis for a topic. 

More here: 

- https://scikit-learn.org/stable/modules/decomposition.html#non-negative-matrix-factorization-nmf-or-nnmf
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

In [ ]:
"""Convert a collection of raw documents to a matrix of TF-IDF features."""

vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(merged_abstracts_list)

In [ ]:
"""Get feature names."""

vectorizer_feature_names = vectorizer.get_feature_names()

In [ ]:
"""Run the model with 100 topics."""

nmf = NMF(n_components=100, verbose=2).fit(tfidf)

In [12]:
"""View the list of topics (10 top words per topic)"""

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

In [ ]:
"""View a top document related to a given topic"""

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(filtered_abstracts[doc_index])

As a result of the first topic model with 100 topics, 2 topics related to opioids and drug abuse are found.

In [10]:
"""Get topic weights per document."""

nmf_W = nmf.transform(tfidf) 

topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[25], i[57]]) # get topic weights for 2 opioid- and drug abuse-related topics

In [ ]:
"""Get those abstracts which have at least some value (not zero) for either of 2 topics."""

topics_list_dataframe = pd.DataFrame(topics_weights)

abstracts = abstracts.reset_index()
topics_list_dataframe = topics_list_dataframe.rename(columns={0:'index'})
concat = pd.concat([abstracts,topics_list_dataframe],axis=1)

filtered = concat[(concat[1] != 0) | (concat[2] != 0)]

In [9]:
"""Get a list of abstracts from the filtered dataframe above."""

filtered_abstracts = filtered[' ABSTRACT'].values.tolist()

In [10]:
len(filtered_abstracts)

416312

In [ ]:
"""Run another topic model with 100 topics on the filtered abstracts."""

"""Convert a collection of raw documents to a matrix of TF-IDF features."""

vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(filtered_abstracts)

"""Get feature names"""
vectorizer_feature_names = vectorizer.get_feature_names()

"""Run the model with 100 topics"""
nmf = NMF(n_components=100, verbose=2).fit(tfidf)

"""View the list of topics (10 top words per topic)"""

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

In [ ]:
"""View a top document related to a given topic"""

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(filtered_abstracts[doc_index])

4 related topics are found in a second round of a topic model with 100 topics.

In [ ]:
"""Get abstracts with 4 related topics."""

topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[11], i[25], i[32], i[43]]) # get topic weights for 4 related topics

topics_weights_dataframe = pd.DataFrame(topics_weights)

"""Sum the weights for all 4 related topics per document."""

topics_weights_dataframe['sum'] = topics_weights_dataframe[1] + topics_weights_dataframe[2] + topics_weights_dataframe[3] + topics_weights_dataframe[4]

In [ ]:
"""Filter by those abstracts who have a summed value of more than 0.01."""

filtered_by_threshold = topics_weights_dataframe[topics_weights_dataframe['sum'] >= 0.01]

"""Merge with the original dataframe. Make sure that level_0 column is in both dataframes to merge on."""

filtered_abstracts = filtered_abstracts.reset_index()
filtered_by_threshold = filtered_by_threshold.rename(columns={0:'level_0'})
filtered_by_threshold_finalized = filtered_by_threshold.merge(filtered_abstracts,on='level_0')

filtered_by_threshold_updated = filtered_by_threshold_finalized[['PROJECT_ID',' ABSTRACT']]

"""Export results to .CSV"""

filtered_by_threshold_updated.to_csv('Filtered_Results_Abstracts.csv')